In [1]:
import os
import torch
import numpy as np
import pandas as pd
import gc

from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, PeftConfig

2023-12-20 23:19:40.583315: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 23:19:40.636035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 23:19:40.636065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 23:19:40.637395: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 23:19:40.644684: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-20 23:19:40.645688: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='train_merged.csv', split='train')

In [3]:
dataset

Dataset({
    features: ['Unnamed: 0', 'CandidateID', 'resume_text', 'target_resume', 'Match Percentage'],
    num_rows: 90
})

In [4]:
def formatting_text_comp_prompts_func(example):
    output_texts = []
    for i in range(len(example['CandidateID'])):
        text = f"Target resume: {example['target_resume'][i]}\n### Input resume: {example['resume_text'][i]}\n### Score: {example['Match Percentage'][i]}"
        output_texts.append(text)
    return output_texts

In [5]:
model_name = "Open-Orca/Mistral-7B-OpenOrca" 

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.pad_token = tokenizer.eos_token

In [9]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=4,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [13]:
training_args = TrainingArguments(
    output_dir="./hidden_adapter/run/",
    # evaluation_strategy="steps",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=64,
    # eval_steps=5,
    save_steps=25,
    logging_steps=5,
    learning_rate=3e-4,
    num_train_epochs=100,
    lr_scheduler_type="cosine",
    warmup_steps=15,
    optim="adamw_torch"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    # eval_dataset=test_dataset,
    formatting_func=formatting_text_comp_prompts_func,
    args=training_args,
    peft_config=peft_config,
    max_seq_length=4096,
    dataset_batch_size=4
    
)

In [15]:
trainer.train()

Step,Training Loss
5,1.004400
10,0.961100
15,0.844900
20,0.652300
25,0.456600
30,0.371900
35,0.344300
40,0.327900
45,0.313100
50,0.287100


TrainOutput(global_step=100, training_loss=0.4094299900531769, metrics={'train_runtime': 10171.6112, 'train_samples_per_second': 0.885, 'train_steps_per_second': 0.01, 'total_flos': 1.7862298023459226e+17, 'train_loss': 0.4094299900531769, 'epoch': 59.0})

In [16]:
trainer.save_model('./hidden_adapter/')